In [1]:
import requests
import asyncio
import nest_asyncio
from datetime import datetime, timedelta
from telegram import Bot

In [2]:
# Enable asyncio in an IPython environment
nest_asyncio.apply()

In [3]:
BOT_TOKEN = '6593706311:AAGfep182h5A1hOifip4095pVEy4Rp7kZTs'
CHAT_ID = "@roofpocazz"

In [4]:
# For Coinbase, symbols might be like 'BTC-USD' instead of 'BTCUSDT'
symbols = ['BTC-USDT', 'ETH-USDT', 'XRP-USDT', 'LTC-USDT', 'ADA-USDT', 'SOL-USDT', 'LINK-USDT', 'XMR-USDT']

In [5]:
COINBASE_API_URL = 'https://api.coinbase.com/v2/prices/'

In [6]:
def get_current_price(symbol):
    url = f'{COINBASE_API_URL}{symbol}/spot'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            price = data['data']['amount']
            return price
        except Exception as e:
            print('Error processing response:', e)
            print(response.content)  # Print the full response for troubleshooting
    else:
        print(f'Failed to fetch data for {symbol}. Status code: {response.status_code}')
        return None

In [7]:
async def send_telegram_message(message):
    bot = Bot(token=BOT_TOKEN)
    await bot.send_message(chat_id=CHAT_ID, text=message)

In [8]:
async def fetch_and_send_prices():
    gmt_plus_6_time = datetime.utcnow() + timedelta(hours=6)
    message = f'COINBASE: Prices at {gmt_plus_6_time.strftime("%Y-%m-%d %H:%M:%S")} (GMT+6):\n\n'
    
    for symbol in symbols:
        price = get_current_price(symbol)
        if price is not None:
            formatted_price = f'{float(price):.2f}'
            message += f'{symbol}: {formatted_price}\n'
    
    await send_telegram_message(message)

In [9]:
# Run the asyncio event loop
async def main():
    await fetch_and_send_prices()

In [10]:
if __name__ == "__main__":
    asyncio.run(main())